# Description

---
.


THIS IS A MONTHLY NOTEBOOK

.

---



This notebook is used to extract ET and other data for analysis in other notebooks/code.

The SEBAL calculations for ET are leveraged from https://github.com/gee-hydro/geeSEBAL

Usage:

  1) Show map with labeled samples for rain or irrigated locations

  2) Manually specify Area of Interest (AOI) with bounding box on map

  3) Specify the filename and number of sample locations to generate within the AOI

  4) Launch batch data extraction


# Setup Notebook

In [ ]:
#!pip install geemap
#!pip install geopandas

In [38]:
import subprocess
from tqdm.notebook import tqdm

In [6]:
import ee
ee.Authenticate()
ee.Initialize(project='second-impact-342800')
import geemap

import numpy as np
import pandas as pd
import datetime
from timeit import default_timer as timer
from datetime import timedelta

Enter verification code: 4/1AfJohXlrsbR3URnY3eacNS2pWHkHNw9X9y-S-k3rRDheIGsxogj0U3PIr7A

Successfully saved authorization token.


In [7]:
# this is used on a cloud server
#%cd /content
#!git clone https://github.com/gee-hydro/geeSEBAL.git
#!git clone https://github.com/earthdataanalytics/irrigation.git
#!cp -r geeSEBAL/etbrasil/ irrigation/
#%cd irrigation/

### Import Custom Libraries

In [8]:
from pipeline import boundaries as bnd
from pipeline import labeledsamples as ls
from pipeline import dataextract as dx
from pipeline import cropmasks as msk

## 1) Show labeled locations on map

Rain = Red = 0

Irrigated = Blue = 1

In [9]:
# labeled MONTHLY data
sample_locations = ls.getRawMonthlyLabeledData()
irr_west = sample_locations.filter(ee.Filter.eq('POINT_TYPE', 1))
rain_west = sample_locations.filter(ee.Filter.eq('POINT_TYPE', 0))

In [10]:
with ee.profilePrinting():
    ee.data.setDefaultWorkloadTag('hydracarta_backend')
    ee.data.setWorkloadTag('hydracarta_view_training_labels')

    Map = geemap.Map()
    Map.add_basemap('HYBRID')
    Map.addLayer(irr_west, { 'color': 'blue'}, 'irrigated', opacity=0.5)
    Map.addLayer(rain_west, { 'color': 'red'}, 'rainfed', opacity=0.5)
    Map.center_object(irr_west)

 EECU·s PeakMem Count  Description
  1.131    517k  1464  Loading assets: projects/eda-bjonesneu-proto/assets/irrigation/labels/(...)
  0.503     68k  2295  (plumbing)
  0.035     48k     1  Listing collection
  0.018    528k     4  Algorithm Collection.geometry
  0.001     16k    12  Algorithm Collection.style
  0.000    4.0k     4  Algorithm Image.visualize
  0.000    4.0k     6  Algorithm Image.updateMask
  0.000    3.2k     6  Algorithm Image.constant
  0.000    4.5k     6  Algorithm Image.blend
  0.000    4.5k     3  Algorithm Geometry.bounds
  0.000    3.3k     3  Algorithm Geometry.transform
   -       751k   549  Algorithm Collection.loadTable
   -       240k   732  Loading assets: projects/eda-bjonesneu-proto/assets/irrigation/labels
   -        14k     9  Algorithm Collection.filter
   -       5.7k     9  Algorithm Collection.map
   -       3.6k  2430  Algorithm Collection.merge
   -       3.3k     9  Algorithm Filter.eq
   -       3.3k     3  Algorithm Element.set
   -      

## 2) Draw a bounding box on the map around the area in which to extract data

In [11]:
Map

Map(center=[37.57832872246989, -120.54743003193794], controls=(WidgetControl(options=['position', 'transparent…

## 3) Generate sample locations within the bounding box

Specify the:

* name of the AOI
* number of sample locations to generate


This displays the generated samples as Yellow points on the map.

In [12]:
# specify manually
aoi_name = '20230925_restart' # used in the saved filename
num_samples = 100#2000 # number of samples for each class

if Map and (len(Map.draw_features) > 0):
    with ee.profilePrinting():
        AOI = ee.FeatureCollection(Map.draw_features)
        sample_locs = ls.generateMonthlySampleLocations(AOI, aoi_name, 
                                                  num_samples=num_samples,
                                                  #start_yr=2015, end_yr=2021,
                                                  excludeNonCrop=True
                                                )
        rain_count = sample_locs.filter(ee.Filter.eq('POINT_TYPE', 0)).size().getInfo()
        irr_count = sample_locs.filter(ee.Filter.eq('POINT_TYPE', 1)).size().getInfo()
        print(f'Number of sample points generated by type: rain={rain_count}, irr={irr_count}')
        Map.addLayer(sample_locs.draw(color="yellow", pointRadius=5, strokeWidth=3), {}, 'sample_locations', opacity=0.5)
        Map.centerObject(sample_locs)

else:
    print("No AOI selected on Map")

Number of sample points generated by type: rain=100, irr=100


 EECU·s PeakMem Count  Description
411.030    197M  7360  Loading GeoTIFF metadata
 58.486    7.0M 83932  (plumbing)
 46.498    585k 27206  Loading assets: projects/eda-bjonesneu-proto/assets/irrigation/labels/(...)
  9.690     59M  5034  no description available
  5.006     18k   348  Algorithm ImageCollection.mosaic
  1.804    1.0M 22084  Algorithm Image.loadGeoTIFF
  1.248     22M    40  Loading GeoTIFF
  0.206     87k   131  Listing collection
  0.147     84k  2208  Algorithm Image.reduceRegions
  0.012    1.7k   144  Algorithm Image.clip computing pixels
  0.008     320    72  Algorithm Image.constant computing pixels
  0.008     360    80  Reprojecting pixels from GEOGCS["WGS 84",DATUM["World Geodetic System 1984",SPHEROID["WGS 84", 637813...] to EPSG:4326
  0.007    1.5k   120  Algorithm Image.loadGeoTIFF computing pixels
  0.006    187k     2  Algorithm Collection.reduceColumns with reducer Reducer.count
  0.005    3.2k   128  Algorithm ReduceRegions.AggregationContainer
  0.00

## 4) Launch batch data extraction

In [13]:
if sample_locs:
    with ee.profilePrinting():
        ee.data.resetWorkloadTag()
        ee.data.setWorkloadTag('hydracarta_export_training_labels')
        export_tasks = dx.extractMonthlyData(sample_locs, aoi_name)#, restart_index=1825)

Number of samples to extract = 193


Number of tasks launched = 193


 EECU·s PeakMem Count  Description
125.032    208M  1840  Loading GeoTIFF metadata
 59.656    7.0M 260610  (plumbing)
 39.604    584k 152011  Loading assets: (...)/(...)
  2.913     59M 13791  no description available
  1.511     17k   280  Algorithm ImageCollection.mosaic
  0.399     22M    10  Loading GeoTIFF
  0.363    1.0M  5714  Algorithm Image.loadGeoTIFF
  0.154    7.5M   122  Algorithm (user-defined function)
  0.088    333k    36  Listing collection
  0.042     81k   745  Algorithm Image.reduceRegions
  0.018    558k   773  Algorithm Geometry.buffer
  0.015    3.2k  2386  Algorithm Element.geometry
  0.003    1.7k    36  Algorithm Image.clip computing pixels
  0.003     336    18  Algorithm Image.constant computing pixels
  0.002    1.4k    30  Algorithm Image.loadGeoTIFF computing pixels
  0.002    5.5k   419  Algorithm Image.clip
  0.002    3.2k    32  Algorithm ReduceRegions.AggregationContainer
  0.001    4.3k  3314  Algorithm Image.constant
  0.001    4.8k   612  Algorith

In [18]:
!earthengine task list --status RUNNING READY | wc -l

      18


In [ ]:
#!earthengine task cancel all

In [33]:
!earthengine task list -l >> task_list_training.txt

In [23]:
with open('task_list_training.txt') as f:
    task_ids = [x.split(' ')[0] for x in f.readlines()]
task_ids

['P4AVET7754HEOG5NOIJYF74F',
 'Z6HLLKNKXFPM3FUIAJYP6AL5',
 'IVHU2XJFEF3ELNONDOFQHEIF',
 'XLT42M6VCUN7VHYC4UH2IIFV',
 'HGVQXPFRBSMK74MZKPPOS4GU',
 '3GK545HIN5VRWKQPTNORUURU',
 '7GC4QBLJCQ4YOW2OU66O5XBS',
 'BWWA7FNRZILGPD2HKREJFEMG',
 'TLR7SGSR54XWIJP7ZQ2RGSSM',
 '3OKL2MRN46XGQO5TB5OIW3TF',
 'OVVTVY2PWVAPRUYJ2ZZWIXN3',
 'USKFJALRLRXKDOTAGZRSQK3H',
 'UVU6HBMLB63CLJNVXIEFDBSG',
 'TBDLTGYO2PEF7V6GK6EXJCA2',
 'YOVZ3O6RPT6GNPQZPR2HY2BM',
 'F4YFEH3JVNI6FRNZIPIFU26H',
 'U6HSUTBJZNDWMLM3KBB6TERZ',
 '3RQPCF6T2PAM3E7EPFZ5H7PM',
 'YB55FFXFOHTMKWBDA2XMHA3H',
 '5VVDAPBTDKS7FLASO4NOIWRQ',
 'GY2KJWZ2CKBE57TFB355UPMB',
 'Q6HZVXZN743Y3LFE3QPPAPG6',
 'UK5TL4CRKBZIRLJYI23IIW2C',
 'TBDTGNFESEX2CYJRPROWDG6V',
 'DQLVUBAEYRCP3E6XTML6G4V4',
 '5HM6DXGEAF4CS22NNDZUW64N',
 '7G2INBRBBRS4OVAXYBE3FYI7',
 'ZCIESPYD3WFK3FC4KKR5PEZW',
 'IHG55YJARUIPO7RLTS76B57L',
 'TIPEDNIBPZ3EFHUOEUVQKHCB',
 '6HWJHEWBQWWHETDUQQ6SJNUO',
 '4Y3HMCSQCJ4FQPT2OOITK6LF',
 'DCXQ2EO7PZF7V5JI5O4BRAJE',
 'NMJ4NCJUYUVRQLUC7JACSGIO',
 'NWH3S2T5SQ2F

In [39]:
all_stats = []

for task_id in tqdm(task_ids):
    task_stats = ee.data.getTaskStatus(task_id)
    all_stats.extend(task_stats)
    
df = pd.DataFrame(all_stats)

  0%|          | 0/193 [00:00<?, ?it/s]

In [65]:
df.to_csv('gee_batch_cost_data_training.csv')

In [41]:
df.head()

,state,description,creation_timestamp_ms,update_timestamp_ms,start_timestamp_ms,task_type,destination_uris,attempt,batch_eecu_usage_seconds,id,name
0,COMPLETED,192,1695662922207,1695670037477,1695670027350,EXPORT_FEATURES,[https://drive.google.com/#folders/19xgV8f73Ee...,1,9.353132,P4AVET7754HEOG5NOIJYF74F,projects/second-impact-342800/operations/P4AVE...
1,COMPLETED,191,1695662919594,1695670008095,1695669997592,EXPORT_FEATURES,[https://drive.google.com/#folders/19xgV8f73Ee...,1,10.531713,Z6HLLKNKXFPM3FUIAJYP6AL5,projects/second-impact-342800/operations/Z6HLL...
2,COMPLETED,190,1695662916949,1695670081259,1695669984194,EXPORT_FEATURES,[https://drive.google.com/#folders/19xgV8f73Ee...,1,6375.973633,IVHU2XJFEF3ELNONDOFQHEIF,projects/second-impact-342800/operations/IVHU2...
3,COMPLETED,189,1695662914298,1695669980078,1695669972526,EXPORT_FEATURES,[https://drive.google.com/#folders/19xgV8f73Ee...,1,15.517467,XLT42M6VCUN7VHYC4UH2IIFV,projects/second-impact-342800/operations/XLT42...
4,COMPLETED,188,1695662911892,1695669971549,1695669957033,EXPORT_FEATURES,[https://drive.google.com/#folders/19xgV8f73Ee...,1,25.353384,HGVQXPFRBSMK74MZKPPOS4GU,projects/second-impact-342800/operations/HGVQX...


In [67]:
total_batch_eecu_secs = df.batch_eecu_usage_seconds.sum()
cost_per_batch_eecu_hour = 0.4 # $0.4 per eecu-hour
total_cost = total_batch_eecu_secs / 3600 * cost_per_batch_eecu_hour

print(f'Total EECU-secs:   {total_batch_eecu_secs:>10,.0f}')
print(f'Total EECU-hours:  {total_batch_eecu_secs / 3600:>10,.2f}')
print(f'Total cost:       ${total_cost:>10,.2f}')

Total EECU-secs:    1,439,163
Total EECU-hours:      399.77
Total cost:       $    159.91
